In [1]:
import sys
sys.path.append("../")

from PIL import Image

import torch
import torchvision
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset

from encoders import resnet18, resnet50
from aug import get_relic_aug_inference

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
IMAGE_SIZE = 32

transform = get_relic_aug_inference()
train_ds = torchvision.datasets.CIFAR10("../data/cifar",
                                  train=True,
                                  transform=transform,
                                  download=True)
val_ds = torchvision.datasets.CIFAR10("../data/cifar",
                                  train=False,
                                  transform=transform,
                                  download=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
len(train_ds), len(val_ds)

(50000, 10000)

In [5]:
# ckpt = torch.load("../models/sota_cifar_2048/encoder.pth")
# ckpt = torch.load("../models/sota_cifar/encoder.pth")
ckpt = torch.load("../models/encoder.pth")

# model = resnet50()
model = resnet18()
model.load_state_dict(ckpt)

model = model.eval().to(device)

In [6]:
from tqdm.auto import tqdm

def get_embs_targets(ds):
    idx = 0
    embs, targets = [], []
    for idx, (image, target) in enumerate(tqdm(ds)):
        with torch.no_grad():
            image = image.unsqueeze(0).to(device)
            out = model(image)
            features = out[0].cpu().detach().tolist()   
            embs.append(features)
            targets.append(target)
    return embs, targets

In [7]:
embs, targets = get_embs_targets(train_ds)
embs_val, targets_val = get_embs_targets(val_ds)

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [8]:
import numpy as np

embeddings = np.array(embs)
labels = np.array(targets)
embeddings_val = np.array(embs_val)
labels_val = np.array(targets_val)

In [9]:
print(embeddings.shape)
print(labels.shape)
print(embeddings_val.shape)
print(labels_val.shape)

(50000, 512)
(50000,)
(10000, 512)
(10000,)


In [10]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
 
X_train, X_test = embeddings, embeddings_val
y_train, y_test = labels, labels_val

print("train", X_train.shape[0], len(y_train))
print("test", X_test.shape[0], len(y_test))
 
clf = LogisticRegression(max_iter=100)
clf = CalibratedClassifierCV(clf)
 
clf.fit(X_train, y_train)
 
y_pred = clf.predict(X_test)
 
acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
 
print("Accuracy: ", acc)
print("Confusion matrix: \n", conf_matrix)
print("Classification report: \n", class_report)
 
y_pred_train = clf.predict(X_train)
class_report = classification_report(y_train, y_pred_train)
print("Classification report train: \n", class_report)

train 50000 50000
test 10000 10000


/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Accuracy:  0.692
Confusion matrix: 
 [[723  17  70  21  19   4  20  14  75  37]
 [ 24 793   8   8   9   6  18   6  41  87]
 [ 58   5 570  65  92  53  91  35  10  21]
 [ 25   6  82 478  70 155 108  39  11  26]
 [ 19   3  77  56 622  38  82  86  14   3]
 [ 11   4  64 164  46 549  80  66   4  12]
 [ 10   4  34  51  28  33 822   9   5   4]
 [ 23   5  29  42  58  46  18 761   3  15]
 [ 68  27  12  18   9   4  11   3 815  33]
 [ 30  80  15  18   3   8  11  15  33 787]]
Classification report: 
               precision    recall  f1-score   support

           0       0.73      0.72      0.73      1000
           1       0.84      0.79      0.82      1000
           2       0.59      0.57      0.58      1000
           3       0.52      0.48      0.50      1000
           4       0.65      0.62      0.64      1000
           5       0.61      0.55      0.58      1000
           6       0.65      0.82      0.73      1000
           7       0.74      0.76      0.75      1000
           8       0

In [10]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
 
X_train, X_test = embeddings, embeddings_val
y_train, y_test = labels, labels_val

print("train", X_train.shape[0], len(y_train))
print("test", X_test.shape[0], len(y_test))
 
clf = LogisticRegression(max_iter=100)
clf = CalibratedClassifierCV(clf)
 
clf.fit(X_train, y_train)
 
y_pred = clf.predict(X_test)
 
acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
 
print("Accuracy: ", acc)
print("Confusion matrix: \n", conf_matrix)
print("Classification report: \n", class_report)
 
y_pred_train = clf.predict(X_train)
class_report = classification_report(y_train, y_pred_train)
print("Classification report train: \n", class_report)

train 50000 50000
test 10000 10000


/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Accuracy:  0.6958
Confusion matrix: 
 [[763  17  50  16  16   6  14   7  72  39]
 [ 27 811   5  10   7   2   7   4  40  87]
 [ 73   6 554  69  86  64  92  33  11  12]
 [ 31   7  80 494  62 151  99  48   7  21]
 [ 17   4  87  54 599  41  91  88  15   4]
 [ 11   8  50 177  48 545  76  71   4  10]
 [  9   5  32  53  34  25 824   6   3   9]
 [ 20   4  30  43  55  60  18 743   5  22]
 [ 61  34  15  13   7   4   9   4 826  27]
 [ 23  78  12  15   4   7   8  19  35 799]]
Classification report: 
               precision    recall  f1-score   support

           0       0.74      0.76      0.75      1000
           1       0.83      0.81      0.82      1000
           2       0.61      0.55      0.58      1000
           3       0.52      0.49      0.51      1000
           4       0.65      0.60      0.62      1000
           5       0.60      0.55      0.57      1000
           6       0.67      0.82      0.74      1000
           7       0.73      0.74      0.73      1000
           8       